In [1]:
#default_exp manifest_gen

In [2]:
#export

def build_volume_manifest(volume_info, manifest_server=None, image_server=None, default_server=None, output_dir_prefix=None):
    '''
    Builds a IIIF manifest for an SSDA volume based on specified metadata.
        volume_id: unique ID for specified volume
        volume_geography: a dictionary containing the volume's location within SSDA geographic hierarchy (used to build S3 url)
        image_details: a dictionary containing the file names, extensions, and dimensions of all images in the volume
        output_dir_prefix: path to append output manifest file name to, if applicable
        manifest_server: base URL for manifest server, including trailing forward slash
        
        returns: path to a json file containing a valid IIIF manifest for the volume
    '''    
    if default_server == None:
        default_server = "https://images.slavesocieties.org/"
    
    if output_dir_prefix != None:
        volume_json_path = output_dir_prefix + '\\' + str(volume_info["identifier"]) + ".json"
    else:
        volume_json_path = str(volume_info["identifier"]) + ".json"   
    
    with open(volume_json_path, "w", encoding = "utf-8") as outfile:
        outfile.write('{\n')
        #add indentation eventually?
        outfile.write("\"@context\": \"http://iiif.io/api/presentation/2/context.json\",\n")
        outfile.write("\"@type\": \"sc:Manifest\",\n")
        outfile.write("\"@id\": \"")
        write_server(outfile, manifest_server, default_server)        
        outfile.write("manifest/" + volume_info["identifier"] + ".json\",\n")
        if ("title" in volume_info) and (volume_info["title"] != None):
            outfile.write("\"label\": \"" + volume_info["title"].replace("\"", "\\\"") + "\",\n")
        else:
            outfile.write("\"label\": \"\",\n")
        if "description" in volume_info:
            outfile.write("\"description\": \"" + volume_info["description"][0].replace("\"", "\\\"").replace("  ", ' ') + "\",\n")
        else:
            outfile.write("\"description\": \"\",\n")
        outfile.write("\"metadata\": [\n")
        outfile.write("{\n")
        outfile.write("\"label\": \"Title\",\n")
        if ("title" in volume_info) and (volume_info["title"] != None):
            outfile.write("\"value\": \"" + volume_info["title"].replace("\"", "\\\"") + "\"\n")
        else:
            outfile.write("\"value\": \"\"\n")
        outfile.write("},\n")        
        if "creator" in volume_info:
            outfile.write("{\n")
            outfile.write("\"label\": \"Creator\",\n")
            outfile.write("\"value\": \"" + volume_info["creator"][0].replace("  ", ' ') + "\"\n")        
            outfile.write("},\n")        
        if ("subject" in volume_info) and (volume_info["subject"] != None):
            vol_subject = ''
            for subject in volume_info["subject"]:
                if len(vol_subject) > 0:
                    vol_subject += '; '
                vol_subject += subject           
            outfile.write("{\n")
            outfile.write("\"label\": \"Subject\",\n")
            outfile.write("\"value\": \"" + vol_subject + "\"\n")
            outfile.write("},\n")
        outfile.write("{\n")
        outfile.write("\"label\": \"Digitized by\",\n")
        if "publisher" in volume_info:
            outfile.write("\"value\": \"" + volume_info["publisher"][0] + "\"\n")
        else:
            outfile.write("\"value\": \"Slave Societies Digital Archive\"\n")
        outfile.write("},\n")
        outfile.write("{\n")
        outfile.write("\"label\": \"Identifier\",\n")
        outfile.write("\"value\": \"" + volume_info["identifier"] + "\"\n")
        outfile.write("},\n")
        if ("date" in volume_info) and (volume_info["date"] != None):
            outfile.write("{\n")
            outfile.write("\"label\": \"Date\",\n")
            outfile.write("\"value\": \"" + volume_info["date"][0] + "\"\n")
            outfile.write("},\n")
        vol_lang = ''
        if "language" in volume_info:            
            for language in volume_info["language"]:
                if len(vol_lang) > 0:
                    vol_lang += '; '
                if language == "spa":
                    vol_lang += "Spanish"
                elif language == "por":
                    vol_lang += "Portuguese"
                elif language == "eng":
                    vol_lang += "English"
                elif language == "lat":
                    vol_lang += "Latin"
                elif language == "fre":
                    vol_lang += "French"
        outfile.write("{\n")
        outfile.write("\"label\": \"Language\",\n")
        outfile.write("\"value\": \"" + vol_lang + "\"\n")
        outfile.write("}\n")
        outfile.write("],\n")            
        
        outfile.write("\"attribution\": \"Slave Societies Digital Archive\",\n")
        outfile.write("\"logo\": \"")
        write_server(outfile, image_server, default_server) 
        outfile.write("iiif/3/ssda_logo_horizontal.jpg/full/max/0/default.jpg\",\n")
        
        outfile.write("\"sequences\": [\n")
        outfile.write("{\n")
        outfile.write("\"@type\": \"sc:Sequence\",\n")
        outfile.write("\"@id\": \"")
        write_server(outfile, manifest_server, default_server)           
        outfile.write("sequence/" + str(volume_info["identifier"]) + ".json\",\n")
        outfile.write("\"canvases\": [\n")
        
        first_image = True
        for image in volume_info["images"]:
            if not first_image:
                outfile.write(",\n")
            else:
                first_image = False
            
            outfile.write("{\n")
            outfile.write("\"@type\": \"sc:Canvas\",\n")            
            outfile.write("\"@id\": \"")
            write_server(outfile, manifest_server, default_server)
            image_number = image["file_name"] - 1000
            image_id = "0" * (4 - len(str(image_number))) + str(image_number)
            canvas_id = str(volume_info["identifier"]) + '-' + image_id
            outfile.write("canvas/" + canvas_id + ".json\",\n")
            outfile.write("\"label\": \"" + str(image_number) + "\",\n")
            outfile.write("\"width\": " + str(image["width"]) + ",\n")
            outfile.write("\"height\": " + str(image["height"]) + ",\n")
            outfile.write("\"images\": [\n")
            outfile.write("{\n")
            outfile.write("\"@type\": \"oa:Annotation\",\n")            
            outfile.write("\"@id\": \"")
            write_server(outfile, manifest_server, default_server)
            outfile.write("annotation/" + str(canvas_id) + ".json\",\n")
            outfile.write("\"motivation\": \"sc:painting\",\n")
            outfile.write("\"on\": \"")
            write_server(outfile, manifest_server, default_server)
            outfile.write("canvas/" + canvas_id + ".json\",\n")
            outfile.write("\"resource\": {\n")            
            outfile.write("\"@type\": \"dctypes:Image\",\n")            
            outfile.write("\"format\": \"image/jpg\",\n")
            outfile.write("\"@id\": \"")
            write_server(outfile, image_server, default_server)
            outfile.write("iiif/3/" + canvas_id + ".jpg/full/max/0/default.jpg\",\n")
            outfile.write("\"width\": " + str(image["width"]) + ",\n")
            outfile.write("\"height\": " + str(image["height"]) + ",\n")
            outfile.write("\"service\": {\n")
            outfile.write("\"@id\": \"")
            write_server(outfile, image_server, default_server)
            outfile.write( "iiif/3/" + canvas_id + ".jpg\",\n")
            outfile.write("\"@context\": \"http://iiif.io/api/image/3/context.json\",\n")
            outfile.write("\"profile\": \"http://iiif.io/api/image/3/level2.json\"\n")
            outfile.write("}\n")
            outfile.write("}\n")
            outfile.write("}\n")
            outfile.write("]\n")
            outfile.write("}")
            
        outfile.write("\n")
        outfile.write("]\n")
        outfile.write("}\n")
        outfile.write("]\n")
        outfile.write("}\n")
    
    return volume_json_path

In [3]:
#export

def write_server(outfile, server, default):
    if server == None:
        outfile.write(default)
    else:
        outfile.write(server)

In [6]:
#no_test

import json

with open("cuba.json", encoding="utf-8") as jsonfile:
    data = json.load(jsonfile)
    
len(data["volumes"])

358

In [7]:
#no_test

for volume in data["volumes"]:
    build_volume_manifest(volume, manifest_server="https://ssda-iiif.s3.amazonaws.com/", image_server="https://images.slavesocieties.org/", default_server="https://images.slavesocieties.org/", output_dir_prefix="manifests")

In [6]:
#no_test

from nbdev.export import notebook2script
notebook2script()

Converted manifest_generation.ipynb.
Converted s3_scrape.ipynb.
Converted s3_scrape_dev.ipynb.
